In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
train = pd.read_csv('data/train.csv', usecols=['Semana','Cliente_ID', 'Producto_ID', 'Demanda_uni_equil'])

In [5]:
train_grouped = train.groupby(['Cliente_ID','Producto_ID'], as_index=False).sum()

In [6]:
train_grouped.head()

,Cliente_ID,Producto_ID,Semana,Demanda_uni_equil
0,26,1182,18,39
1,26,4767,18,126
2,26,30235,9,96
3,26,30314,7,48
4,26,31393,42,119


### Calculations of average demanda (over 7 weeks) for each pair of Cliente and Producto

In [7]:
train_grouped.Demanda_uni_equil = train_grouped.Demanda_uni_equil/7

In [8]:
train_grouped.head()

,Cliente_ID,Producto_ID,Semana,Demanda_uni_equil
0,26,1182,18,5.571429
1,26,4767,18,18.000000
2,26,30235,9,13.714286
3,26,30314,7,6.857143
4,26,31393,42,17.000000


#### Save data of cliente, producto and avg demand to a CSV file, ~ 800 MB

In [ ]:
train_grouped.to_csv('data/train_grouped_ClientProduct_avg.csv')
train_grouped.to_pickle('data/train_grouped_ClientProduct_avg.pkl')

In [9]:
train_grouped.shape

(25831538, 4)

##### Loading the cpd file (stands for cliente_producto_demanda relationship)

In [10]:
cpd = pd.read_csv('data/train_grouped_ClientProduct_avg.csv')

In [11]:
print ('In cpd.csv, there are: ')
for col in cpd.columns:
     print ('{:>10}{:>20}'.format(len(cpd[col].value_counts()), col))

In cpd.csv, there are: 
  25831538          Unnamed: 0
    880604          Cliente_ID
      1799         Producto_ID
       433              Semana
      4651   Demanda_uni_equil


In [12]:
cpd.shape

(25831538, 5)

In [13]:
cpd.head()

,Unnamed: 0,Cliente_ID,Producto_ID,Semana,Demanda_uni_equil
0,0,26,1182,18,5.571429
1,1,26,4767,18,18.000000
2,2,26,30235,9,13.714286
3,3,26,30314,7,6.857143
4,4,26,31393,42,17.000000


In [14]:
test = pd.read_csv('data/test.csv')

In [15]:
test.shape

(6999251, 7)

In [16]:
print ('In test.csv, there are: ')
for col in test.columns:
     print ('{:>10}{:>20}'.format(len(test[col].value_counts()), col))

In test.csv, there are: 
   6999251                  id
         2              Semana
       552          Agencia_ID
         9            Canal_ID
      2608            Ruta_SAK
    745164          Cliente_ID
      1522         Producto_ID


In [17]:
train_prod_unique = set(cpd['Producto_ID'].unique())
train_client_unique = set(cpd['Cliente_ID'].unique())
test_prod_unique = set(test['Producto_ID'].unique())
test_client_unique = set(test['Cliente_ID'].unique())

In [18]:
print(len(test_prod_unique.difference(train_prod_unique)))
print(len(test_client_unique.difference(train_client_unique)))

34
9663


In [19]:
prod_dict = {}
for pid, p in enumerate(train_prod_unique):
        prod_dict[p] = pid

print(len(prod_dict))

client_dict = {}
for cid, c in enumerate(train_client_unique):
        client_dict[c] = cid

print(len(client_dict))

1799
880604


In [20]:
import numpy as np

In [21]:
# client_prod_matrix = np.zeros((len(train_client_unique), len(train_prod_unique)))
client_prod_matrix = np.zeros((1000000, 1000000))

for df_idx, row in cpd.iterrows():
    if df_idx > 1000000:
        break
    m = client_dict[row['Cliente_ID']]
    n = prod_dict[row['Producto_ID']]
    client_prod_matrix[m,n] = row['Demanda_uni_equil']

In [10]:
client_prod_matrix.shape

(1000000, 1000000)

In [ ]:
from scipy import sparse

sparse_client_prod_matrix = sparse.csr_matrix(client_prod_matrix)

In [ ]:
np.save('data/client_prod_matrix.pkl', client_prod_matrix)

In [ ]:
covarian_matrix = np.cov(client_prod_matrix)

In [ ]:
np.save('data/matrix.npy',client_prod_matrix )